In [1]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


/Users/kakao/PycharmProjects/wade.song/.venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",
               "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
               "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
for doc in docs:
    print(doc.page_content[:100])  # 각 문서의 첫 100자를 출력



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |


      Prompt Engineering
    
Date: March 15, 2023  |  Estimated Reading Time: 21 min  |  Author: 


      Adversarial Attacks on LLMs
    
Date: October 25, 2023  |  Estimated Reading Time: 33 min  


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
splits[-10:]

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/'}, page_content='Inner maximization: find the most effective adversarial data point, $\\mathbf{x} + \\boldsymbol{\\delta}$, that leads to high loss. All the adversarial attack methods eventually come down to ways to maximize the loss in the inner loop.\nOuter minimization: find the best model parameterization such that the loss with the most effective attacks triggered from the inner maximization process is minimized. Naive way to train a robust model is to replace each data point with their perturbed versions, which can be multiple adversarial variants of one data point.\n\n\nFig. 17. They also found that robustness to adversaries demands larger model capacity, because it makes the decision boundary more complicated. Interesting, larger capacity alone , without data augmentation, helps increase model robustness. (Image source: Madry et al. 2017)\nSome work on LLM Robustness#\n\nDisclaimer: Not

In [6]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [7]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

In [13]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.25}
)
retrieved_docs = retriever.invoke("agent memory")

In [33]:
def format_docs(docs):
    formated = "\n\n".join(doc.page_content for doc in docs)
    return formated

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [99]:
from langchain_core.prompts import ChatPromptTemplate


#User query와 retrieved chunk 에 대해 relevance 가 있는지를 평가하는 시스템 프롬프트 작성
def prompt_relevance():
    res = '''
**System Prompt:**

You are provided with a User query and a retrieved chunk of text. Your task is to evaluate whether the retrieved chunk is relevant to the User query. 

1. **User Query:** 
```
{question}
```

2. **Retrieved Chunk:** 
```
{context}
```


Please analyze the retrieved chunk and determine if it is relevant to the User query. 

If the chunk is relevant to the query, output: 
{{"relevance": "yes"}}

If the chunk is not relevant to the query, output: 
{{"relevance": "no"}}

Please ensure your evaluation is based on the relevance of the content of the retrieved chunk in relation to the User query.
    '''
    return res




In [105]:
from langchain_core.prompts import PromptTemplate
retriever = vectorstore.as_retriever()
retrieved_docs2 = retriever.invoke("What is decomposition?")
for retrieved_doc in retrieved_docs2:
    prompt_relevance_res = prompt_relevance()
    prompt = PromptTemplate(
        template=prompt_relevance(),
        input_variables=["question", "context"]
    )
    

    chain = RunnablePassthrough() | prompt | llm | JsonOutputParser()
    result = chain.invoke({"question": "What is decomposition?", "context": retrieved_doc.page_content})
    

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
{'relevance': 'no'}
Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks

In [117]:
def check_relevance(question, retrieved_chunk):
    prompt = PromptTemplate(
        template=prompt_relevance(),
        input_variables=["question", "context"]
    )


    chain = RunnablePassthrough() | prompt | llm | JsonOutputParser()
    result = chain.invoke({"question": question, "context": retrieved_chunk})
    return result.get("relevance") == "yes"

def invoke_and_filter_related_docs(question):
    retrieved_docs2 = retriever.invoke(question)
    res = []
    for retrieved_doc in retrieved_docs2:
        if check_relevance(question, retrieved_doc.page_content):
            res.append(retrieved_doc.page_content)
    return res

In [118]:
print(invoke_and_filter_related_docs("What is decomposition?"))

['Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.']


In [132]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc for doc in docs)


from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

# question = 
rag_chain = (
        {"context": lambda question: format_docs(invoke_and_filter_related_docs(question)), "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

"Task decomposition is the process of breaking down a complex task into smaller, manageable steps. It can be achieved through various methods, such as prompting a language model (LLM) for subgoals, using task-specific instructions, or incorporating human inputs. This approach enhances the model's performance by allowing it to think through the task step by step."